# CS542 Project Submission
Team: Lehland Ling, Evan Powell

# Dependencies

In [ ]:
!pip install transformers

ERROR: huggingface-hub 0.14.1 has requirement packaging>=20.9, but you'll have packaging 20.4 which is incompatible.



  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.2
    Uninstalling typing-extensions-3.7.4.2:


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\__pycache__\\typing_extensions.cpython-38.pyc'
Consider using the `--user` option or check the permissions.

ERROR: huggingface-hub 0.14.1 has requirement packaging>=20.9, but you'll have packaging 20.4 which is incompatible.
ERROR: huggingface-hub 0.14.1 has requirement typing-extensions>=3.7.4.3, but you'll have typing-extensions 3.7.4.2 which is incompatible.


  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)


In [ ]:
!pip install torch torchvision torchtext -f https://download.pytorch.org/whl/cpu/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-2.0.0%2Bcpu-cp38-cp38-win_amd64.whl (174.0 MB)


In [ ]:
import os
import json
import pickle
import numpy as np

import pandas as pd
import torch
from types import SimpleNamespace
from transformers import AutoModel, AutoTokenizer, AutoModelForMultipleChoice, AutoModelForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForMultipleChoice, DistilBertForSequenceClassification
import torch.nn.functional as F
from scipy.special import softmax
from torch.utils.data import Dataset, DataLoader

ImportError: cannot import name 'TypeAlias' from 'typing_extensions' (C:\ProgramData\Anaconda3\lib\site-packages\typing_extensions.py)

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
autocast_questions = json.load(open('/content/drive/MyDrive/CS542/autocast_questions.json', encoding="utf-8")) # from the Autocast dataset
test_questions = json.load(open('/content/drive/MyDrive/CS542/autocast_competition_test_set.json', encoding="utf-8"))
test_ids = [q['id'] for q in test_questions]

# Load Models Finetuned on the Training Set

## Random Baseline

In [ ]:
def random_baseline_model(question):
    if question['qtype'] == 't/f':
        return np.random.random(size=2)
    elif question['qtype'] == 'mc':
        probs = np.random.random(size=len(question['choices']))
        return probs / probs.sum()
    elif question['qtype'] == 'num':
        return np.random.random()


def calibrated_random_baseline_model(question):
    if question['qtype'] == 't/f':
        pred_idx = np.argmax(np.random.random(size=2))
        pred = np.ones(2)
        pred[pred_idx] += 1e-5
        return pred / pred.sum()
    elif question['qtype'] == 'mc':
        pred_idx = np.argmax(np.random.random(size=len(question['choices'])))
        pred = np.ones(len(question['choices']))
        pred[pred_idx] += 1e-5
        return pred / pred.sum()
    elif question['qtype'] == 'num':
        return 0.5

## Multiple Choice Forecaster
to use the multiple choice forecaster, run all of the cells in this block and call `mc_predict(test_questions)` on the entire test set

In [ ]:
def mc_preptest(mc_test):
    """takes as input a dataframe containing the multiple choice questions and does the first step toward preprocessing"""

    mc_test['len background'] = mc_test['background'].str.len()
    mc_test['len question'] = mc_test['question'].str.len()
    mc_test['len answers'] = mc_test['choices'].apply(lambda x : [len(ans) for ans in x])
    mc_test['len answers'] = mc_test['len answers'].apply(lambda x: max(x))
    x = len(" [sep] ")
    mc_test['temp'] = mc_test['len background'] - x*2 - mc_test['len answers'] - mc_test['len question']
    mc_test['max context length'] = ((-1 * (mc_test['len answers'] + mc_test['len question']) + x*2) + 512)

    test = mc_test['temp'].compare(mc_test['max context length'])
    mc_test['max context length'] = test.apply(lambda row: row.min(), axis=1)
    mc_test['prompt'] = mc_test.apply(lambda x: x['background'][:x['max context length']], axis=1)
    mc_test['len prompt'] = mc_test['prompt'].str.len()
    mc_test['len prompt'].max()

    #return data
    return mc_test

In [ ]:
def mc_preprocess_candidates(row):
  """preprocesses a row of the multiple choice questions to be tokenized"""
  choices = []
  prompt =  " [SEP] ".join([row['prompt'], row['question']]) + " [SEP] "

  for choice in row['choices']:
        # text = item['sent1'] + " " + item['sent2'] + " " + item[f'ending{i}']
        choices.append([prompt, choice])
  return choices

In [ ]:
#load the distilbert tokenizer and the fine-tuned model
mc_tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
mc_model = AutoModelForMultipleChoice.from_pretrained(r"/content/drive/MyDrive/CS542/mcfor")

In [ ]:
def mc_predict(test_questions):
  """given the test questions, make predictions on the multiple choice question in the set.
  This function will return a dictionary, where the keys are question ids and the values are 
  predictions"""

  df_test = pd.DataFrame(test_questions)
  mc_test = df_test.loc[df_test['qtype'] == 'mc']
  mc_test_data = mc_preptest(mc_test)

  predictions = {}
  probabilities = {}

  for i in range(len(mc_test_data)):
    this_question = mc_preprocess_candidates(mc_test_data.iloc[i])

    inputs = mc_tokenizer(this_question, return_tensors="pt", padding=True)
    labels = torch.tensor(0).unsqueeze(0)

    outputs = mc_model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels = labels)
    logits = outputs.logits

    probs = F.softmax(logits, dim=-1).squeeze().tolist()
    predicted_class = logits.argmax().item()

    question_id = mc_test_data.iloc[i]["id"]

    if question_id in predictions:
      while question_id in predictions and len(question_id) < 11:
        question_id += "a"

    predictions[question_id] = predicted_class
    probabilities[question_id] = probs

  return probabilities


## True / False Forecaster

In [ ]:
def prepare_truef_set(test_truef):
  """ prepares a dataframe test_truef for the true / false forecaster """
  test_truef['len background'] = test_truef['background'].str.len()
  test_truef['len question'] = test_truef['question'].str.len()

  test_truef['total prompt'] = test_truef['question'] + ' ' + test_truef['background']

  test_truef['len prompt'] = test_truef['total prompt'].str.len()

  test_truef['total prompt'] = test_truef.apply(lambda x : x['total prompt'][0:511] if x['len prompt'] > 512 else x['total prompt'], axis = 1)

  test_truef['total prompt'].str.len().max

  return test_truef

In [ ]:
#load the distilbert-base-uncased tokenizer
truef_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# load the fine-tuned model
checkpoint_dir = "/content/drive/MyDrive/CS542/tf_checkpoints"
truef_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir)

In [ ]:
class GoodJudgementDatasetTest(torch.utils.data.Dataset): 
  #Class for the true / false subset
    def __init__(self, encodings, labels, ids, split="test", val_split=0.1):
        self.encodings = encodings
        self.labels = labels
        self.ids = ids
        self.split = split
        self.val_split = val_split #only needed for training the model
        self.indices = self.generate_indices()

    def __getitem__(self, idx):
        index = self.indices[idx]
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        item["id"] = self.ids[idx]
        if self.labels is not None:
          item['labels'] = torch.tensor(self.labels[index])
        return item

    def __len__(self):
        return len(self.indices)

    def get_labels(self):
        return [self.labels[i] for i in self.indices]

    def generate_indices(self):
      if self.split == "test":
        return np.arange(len(self.encodings['input_ids']))
      else:
        indices = np.arange(len(self.labels))
        np.random.shuffle(indices)
        split_idx = int(self.val_split * len(indices))

        if self.split == "train":
            return indices[split_idx:]
        elif self.split == "val":
            return indices[:split_idx]


In [ ]:
def truef_predict(test_questions):
  """given the test questions, make predictions on the true / false questions in the set.
  This function will return a dictionary, where the keys are the question ids and the values
  are the prediction values"""

  test_df = pd.DataFrame(test_questions)
  test_truef = test_df.loc[test_df['qtype'] == 't/f']

  

  test_truef = prepare_truef_set(test_truef)

  test_ids_truef = test_truef['id'].tolist()
  test_sentences_truef = test_truef['total prompt'].values.tolist()
  test_encodings_truef = truef_tokenizer(test_sentences_truef, truncation=True, padding=True, return_tensors='pt')

  test_dataset_truef = GoodJudgementDatasetTest(test_encodings_truef, None, test_ids_truef, split="test")

  truef_model.eval()

  predictions = {}
  probabilities_dict = {}

  for i in range(len(test_dataset_truef)):
    input_ids = test_dataset_truef[i]['input_ids'].unsqueeze(0)
    attention_mask = test_dataset_truef[i]['attention_mask'].unsqueeze(0)
    inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

    if 'token_type_ids' in test_dataset_truef[i]:
      token_type_ids = test_dataset_truef[i]['token_type_ids'].unsqueeze(0)
      inputs['token_type_ids'] = token_type_ids

    # Make a prediction
    with torch.no_grad():
      outputs = truef_model(**inputs)

    # Process the output to obtain the prediction
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()

    question_id = test_dataset_truef[i]["id"]

    if question_id in predictions:
      while question_id in predictions and len(question_id) < 11:
        question_id += "a"

    predictions[ question_id ] =  predicted_class
    probabilities_dict[question_id] = softmax(logits.cpu().numpy(), axis=-1).tolist()[0]

  return probabilities_dict

## Numeric Forecaster

In [ ]:
num_model_path = "/content/drive/MyDrive/CS542/numforecast" 
num_model = DistilBertForSequenceClassification.from_pretrained(num_model_path)
num_tokenizer = tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def num_preptesting(mc):
    """modified form of preptrainig - commented out the lines that cannot be done to the test set"""    

    mc['prompt'] = mc['dtype'].apply(lambda x: "The data given is in the units: " + str(x) + " [SEP] " if x is not None else "The data is given only from zero to one [SEP] ")

    return mc

In [ ]:
class NumTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, ids):
        self.encodings = encodings
        self.ids = ids

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['id'] = self.ids[idx]
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
def predict_single_datapoint(model, input_text, tokenizer):
    # Put the model in evaluation mode
    model.eval()

    # Tokenize the input text using the provided tokenizer
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

    # Move the input tensors to the device (either GPU or CPU)
    input_ids = inputs["input_ids"] #.to(device)
    attention_mask = inputs["attention_mask"] #.to(device)

    with torch.no_grad():
        # Forward pass through the model to get the logits
        outputs = model(input_ids, attention_mask=attention_mask)

        # Get the predicted logits
        logits = outputs.logits

        # Detach and move the logits to the CPU
        logits = logits.detach().cpu().numpy()

    # Return the regression value
    return logits[0][0]

In [ ]:
def num_predict(test_questions):
  """given the test questions, make predictions on the numeric questions in the set.
  This function will return a dictionary, where the keys are the question ids and the values
  are the prediction values"""
  df_test = pd.DataFrame(test_questions)
  num_test = df_test.loc[df_test['qtype'] == 'num']

  num_test['dtype'] = num_test['choices'].apply(lambda x: type(x['max']) if x is not None else None)

  num_test_data = num_preptesting(num_test)
  num_test_dataholder = num_test_data[['id', 'prompt']]

  num_test_encodings = tokenizer(num_test_dataholder['prompt'].tolist(), truncation=True, padding=True)

  num_test_dataset = NumTestDataset(num_test_encodings, num_test_dataholder['id'].tolist())
  num_test_dataloader = DataLoader(num_test_dataset, batch_size=1)

  predictions = {}

  for batch in num_test_dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    data_id = batch['id'][0]
    input_text = tokenizer.decode(input_ids.squeeze())
    prediction = predict_single_datapoint(num_model, input_text, num_tokenizer)
    predictions[data_id] = prediction

  return predictions

# Evaluate the model

In [ ]:
tf_results, mc_results, num_results = [],[],[]
for p, a, qtype in zip(preds, answers, qtypes):
    if qtype == 't/f':
        tf_results.append(brier_score(p, a))
    elif qtype == 'mc':
        mc_results.append(brier_score(p, a))
    else:
        num_results.append(np.abs(p - a))

print(f"T/F: {np.mean(tf_results)*100:.2f}, MCQ: {np.mean(mc_results)*100:.2f}, NUM: {np.mean(num_results)*100:.2f}")
print(f"Combined Metric: {(np.mean(tf_results) + np.mean(mc_results) + np.mean(num_results))*100:.2f}")

T/F: 25.00, MCQ: 38.05, NUM: 22.63
Combined Metric: 85.67


# Make predictions on test set

In [ ]:
old_preds = []
for question in test_questions:
    old_preds.append(calibrated_random_baseline_model(question))


In [ ]:
len(old_preds)

1364

In [ ]:
preds = []
id_list = list(map(str, [question['id'] for question in test_questions]))


In [ ]:
truef_preds = truef_predict(test_questions)


<ipython-input-22-63a2175f3ea6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_truef['len background'] = test_truef['background'].str.len()
<ipython-input-22-63a2175f3ea6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_truef['len question'] = test_truef['question'].str.len()
<ipython-input-22-63a2175f3ea6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
mc_preds = mc_predict(test_questions)

<ipython-input-17-ae4045ed9b52>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mc_test['len background'] = mc_test['background'].str.len()
<ipython-input-17-ae4045ed9b52>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mc_test['len question'] = mc_test['question'].str.len()
<ipython-input-17-ae4045ed9b52>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [ ]:
num_preds = num_predict(test_questions)

<ipython-input-53-65baf36a7c51>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_test['dtype'] = num_test['choices'].apply(lambda x: type(x['max']) if x is not None else None)
<ipython-input-28-5197705ffb65>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mc['prompt'] = mc['dtype'].apply(lambda x: "The data given is in the units: " + str(x) + " [SEP] " if x is not None else "The data is given only from zero to one [SEP] ")


In [ ]:
merged_preds = {}
merged_preds.update(truef_preds)
merged_preds.update(mc_preds)
merged_preds.update(num_preds)

In [ ]:
print(len(test_questions))
print(len(truef_preds))
print(len(mc_preds))
print(len(num_preds))
print(len(merged_preds))

1364
826
182
356
1364


In [ ]:
preds = []
for i in range(len(id_list)):
  this_id = id_list[i]
  if this_id in id_list[:i]:
    while this_id in id_list[:i] and len(this_id) < 11: #there are multiple questions that share the same id
      this_id += "a"

  if this_id not in merged_preds:
    print("Not in merged_preds, id: " + this_id)
  else:
    preds += [merged_preds[this_id]]


In [ ]:
len(preds)

1364

In [ ]:
if not os.path.exists('submission'):
    os.makedirs('submission')

with open(os.path.join('submission', 'predictions.pkl'), 'wb') as f:
    pickle.dump(preds, f, protocol=2)

!cd submission && zip ../submission.zip ./* && cd ..

updating: predictions.pkl (deflated 63%)


In [ ]:
!dir

drive  sample_data  submission	submission.zip
